In [1]:
# Test for GPU
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13954228644013493691
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 10828120064
locality {
  bus_id: 1
}
incarnation: 4833219926466238166
physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0"
]


# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [2]:
import helper


# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [3]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [4]:
import collections


english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


In [5]:
print('100 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(100)))[0]) + '"')

100 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois" "jamais" "le" "l'" "généralement" "moins" "au" "aimé" "fruit" "préféré" "agréable" "froid" "son" "chaud" "de" "plus" "automne" "mois" "à" "elle" "citrons" "paris" "inde" "états-unis" "france" "jersey" "new" "chine" "pendant" "pamplemousse" "mon" "votre" "juin" "printemps" "janvier" "hiver" "mars" "été" "mai" "septembre" "juillet" "avril" "novembre" "décembre" "février" "octobre" "aime" "août" "merveilleux" "relaxant" "doux" "humide" "notre" "californie" "sec" "leur" "occupé" "pluvieux" "calme" "beau" "habituellement" "pommes" "pêches" "oranges" "poires" "fraises" "bananes" "verts" "raisins" "mangues" "d'" "mangue" "gel" "raisin" "pomme" "l'orange" "citron" "chaux" "banane" "poire" "fraise" "pêche" "pas" "enneigée" "favori" "déteste" "gèle" "fruits" "voiture" "l'automne" "ils"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [6]:
import project_tests as tests
from keras.preprocessing.text import Tokenizer


def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # DONE: Implement
    tokenizer = Tokenizer(num_words=None,
                                   filters= '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                   lower=True,
                                   split=" ",
                                   char_level=False)
    tokenizer.fit_on_texts(x)
    
    return tokenizer.texts_to_sequences(x), tokenizer
tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence about l\'automne .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21, 'about': 22, "l'automne": 23}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence about l'automne .
  Output: [18, 19, 3, 20, 21, 22, 23]


Using TensorFlow backend.


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [7]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences


def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # DONE: Implement
    return pad_sequences(x,maxlen=length,padding='post')
tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21 22 23]
  Output: [18 19  3 20 21 22 23  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [8]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)

print('Data Preprocessed')

Data Preprocessed


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [9]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])
def logits_to_text2(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in logits])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [10]:
from keras.layers import GRU, SimpleRNN, LSTM, Input, Dense, TimeDistributed
from keras.models import Model
from keras.layers import Activation
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy


def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    RNN = GRU # SimpleRNN, LSTM
    
    input_layer = Input(shape=(input_shape[1],input_shape[2]))
    use_td = True # Time Distribution

    if not use_td:
        rnn_layer = SimpleRNN(units=french_vocab_size, return_sequences=True,activation='softmax',dropout=0.5,implementation=2,unroll=True)(input_layer)
        model = Model(inputs=input_layer,outputs=rnn_layer)
    else:
        # Time-Distributed Method
        rnn_layer = RNN(units=french_vocab_size, return_sequences=True,dropout=0.5,implementation=2,unroll=True)(input_layer)
        timedense_layer = TimeDistributed(Dense(french_vocab_size))(rnn_layer)
        act_layer = Activation('softmax')(timedense_layer)
        model = Model(inputs=input_layer,outputs=act_layer)

    model.summary()
    learning_rate = 0.01
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate,decay=1./100e3),
                  metrics=['accuracy'])
    return model
tests.test_simple_model(simple_model)


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1, # padding adds an additional index
    len(french_tokenizer.word_index)+1) # padding adds an additional index

print ("Training x.shape=",tmp_x.shape," y.shape=",preproc_french_sentences.shape)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 21, 1)             0         
_________________________________________________________________
gru_1 (GRU)                  (None, 21, 344)           357072    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 21, 344)           118680    
_________________________________________________________________
activation_1 (Activation)    (None, 21, 344)           0         
Total params: 475,752.0
Trainable params: 475,752.0
Non-trainable params: 0.0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 21, 1)             0         
________________________________________________________________

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [11]:
from keras.layers.embeddings import Embedding

def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """

    input_layer = Input(shape=(input_shape[1],)) # Comma makes it a tuple
    embed_layer = Embedding(english_vocab_size, french_vocab_size, mask_zero=False,
                            input_length=input_shape[1])(input_layer)
    RNN = GRU # SimpleRNN, LSTM
    rnn_layer = RNN(units=french_vocab_size, return_sequences=True,
                    dropout=0.5,implementation=2,unroll=True)(embed_layer)
    timedense_layer = TimeDistributed(Dense(french_vocab_size))(rnn_layer)
    act_layer = Activation('softmax')(timedense_layer)
    
    
    model = Model(inputs=input_layer,outputs=act_layer)
    model.summary()
    learning_rate = 0.025
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate,decay=1./100e3),
                  metrics=['accuracy'])
    return model
tests.test_embed_model(embed_model)


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))  # New reshape to match test input

# Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1, # padding adds an additional index
    len(french_tokenizer.word_index)+1) # padding adds an additional index

print ("Training x.shape=",tmp_x.shape," y.shape=",preproc_french_sentences.shape)
embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
simple_list = np.squeeze(tmp_x[:1].tolist()).tolist()
print("English:",logits_to_text2(simple_list, english_tokenizer))
print("French:",logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 21)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 21, 344)           68456     
_________________________________________________________________
gru_3 (GRU)                  (None, 21, 344)           711048    
_________________________________________________________________
time_distributed_3 (TimeDist (None, 21, 344)           118680    
_________________________________________________________________
activation_3 (Activation)    (None, 21, 344)           0         
Total params: 898,184.0
Trainable params: 898,184.0
Non-trainable params: 0.0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #  

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [12]:
from keras.layers import Bidirectional


def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    RNN = GRU # SimpleRNN, LSTM
    use_td = True # Time Distribution

    input_layer = Input(shape=(input_shape[1],input_shape[2]))

    if not use_td:
        rnn_layer = Bidirectional(
            RNN(units=french_vocab_size, return_sequences=True,
                activation='softmax',dropout=0.5,implementation=2,unroll=True)
            ,merge_mode='sum'
        )(input_layer)
        model = Model(inputs=input_layer,outputs=rnn_layer)
    else:
        rnn_layer = Bidirectional(
            RNN(units=french_vocab_size, return_sequences=True,
                        dropout=0.5,implementation=2,unroll=True)
            ,merge_mode='sum'
        )(input_layer)

        timedense_layer = TimeDistributed(Dense(french_vocab_size))(rnn_layer)
        act_layer = Activation('softmax')(timedense_layer)
        model = Model(inputs=input_layer,outputs=act_layer)

    model.summary()
    learning_rate = 0.025
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate,decay=1./100e3),
                  metrics=['accuracy'])
    
    return model

tests.test_bd_model(bd_model)

# Reshaping the input to work with a no-embedding RNN
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
bd_rnn_model = bd_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1, # padding adds an additional index
    len(french_tokenizer.word_index)+1) # padding adds an additional index

print ("Training x.shape=",tmp_x.shape," y.shape=",preproc_french_sentences.shape)
bd_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(bd_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 21, 1)             0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 21, 344)           714144    
_________________________________________________________________
time_distributed_5 (TimeDist (None, 21, 344)           118680    
_________________________________________________________________
activation_5 (Activation)    (None, 21, 344)           0         
Total params: 832,824.0
Trainable params: 832,824.0
Non-trainable params: 0.0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 21, 1)             0         
________________________________________________________________

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [21]:
from keras.layers import RepeatVector


def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    hidden_size = 32
    RNN = GRU # SimpleRNN, LSTM

    input_layer = Input(shape=(input_shape[1],input_shape[2]))

    encode_layer = RNN(units=hidden_size, return_sequences=False,
            dropout=0.5,implementation=2,unroll=True)(input_layer)
    
    repeat_layer = RepeatVector(output_sequence_length)(encode_layer)
    
    decode_layer = RNN(units=hidden_size, return_sequences=True,
            dropout=0.5,implementation=2,unroll=True)(repeat_layer)

    timedense_layer = TimeDistributed(Dense(french_vocab_size))(decode_layer)
    act_layer = Activation('softmax')(timedense_layer)
    
    model = Model(inputs=input_layer,outputs=act_layer)
    model.summary()
    learning_rate = 0.025
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate,decay=1./100e3),
                  metrics=['accuracy'])
    
    return model

tests.test_encdec_model(encdec_model)


# Reshaping the input to work with a basic RNN
tmp_x = preproc_english_sentences
tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[-1], 1))

print ("Training x.shape=",tmp_x.shape," y.shape=",preproc_french_sentences.shape)

# Train the neural network
encdec_rnn_model = encdec_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1, # padding adds an additional index
    len(french_tokenizer.word_index)+1) # padding adds an additional index

encdec_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(encdec_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        (None, 15, 1)             0         
_________________________________________________________________
gru_35 (GRU)                 (None, 32)                3264      
_________________________________________________________________
repeat_vector_15 (RepeatVect (None, 21, 32)            0         
_________________________________________________________________
gru_36 (GRU)                 (None, 21, 32)            6240      
_________________________________________________________________
time_distributed_21 (TimeDis (None, 21, 344)           11352     
_________________________________________________________________
activation_21 (Activation)   (None, 21, 344)           0         
Total params: 20,856.0
Trainable params: 20,856.0
Non-trainable params: 0.0
_________________________________________________________________


### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [14]:

def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    
    Structured inspired by:
      Francois Chollet
      "An implementation of sequence to sequence learning for performing addition"
      https://github.com/fchollet/keras/blob/master/examples/addition_rnn.py
    """
    
    embedding_size = 128 #english_vocab_size
    hidden_size = 64 #french_vocab_size
    num_decoder_layers = 1
    RNN = GRU # SimpleRNN, LSTM

    input_layer = Input(shape=(input_shape[1],)) # Comma makes it a tuple
    embed_layer = Embedding(english_vocab_size, embedding_size, mask_zero=False,
                            input_length=input_shape[-1])(input_layer)

    encode_layer = Bidirectional(
        RNN(units=hidden_size, return_sequences=False,
                  dropout=0.5,implementation=2,unroll=True,activation='linear')
        ,merge_mode='sum'
    )(embed_layer) #(repeat_layer)
    repeat_layer = RepeatVector(output_sequence_length)(encode_layer)
    decode_layers = [repeat_layer]
    
    for n in range(num_decoder_layers):
        decode_layers.append( Bidirectional(
            RNN(units=hidden_size, return_sequences=True,
                      dropout=0.5,implementation=2,unroll=True,activation='linear')
            ,merge_mode='sum'
        )(decode_layers[-1]))

    timedense_layer = TimeDistributed(Dense(french_vocab_size))(decode_layers[-1])
    act_layer = Activation('softmax')(timedense_layer)

    model = Model(inputs=input_layer,outputs=act_layer)
    model.summary()
    learning_rate = 0.01
    
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate), #,decay=1./100e3),
                  metrics=['accuracy'])
    
    return model

tests.test_model_final(model_final)


print('Final Model Loaded')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 15)                0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 15, 128)           25472     
_________________________________________________________________
bidirectional_3 (Bidirection (None, 64)                74112     
_________________________________________________________________
repeat_vector_3 (RepeatVecto (None, 21, 64)            0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 21, 64)            49536     
_________________________________________________________________
time_distributed_9 (TimeDist (None, 21, 344)           22360     
_________________________________________________________________
activation_9 (Activation)    (None, 21, 344)           0         
Total para

## Prediction (IMPLEMENTATION)

In [15]:
import numpy as np
from keras.preprocessing.sequence import pad_sequences


def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """

    # No need to reshape english sentences to french sentence sizes

    # Train the neural network
    model = model_final(
            x.shape,
            y.shape[1],
            len(x_tk.word_index)+1, # padding adds an additional index
            len(y_tk.word_index)+1) # padding adds an additional index

    model.fit(x, y, batch_size=1024, epochs=10, validation_split=0.2)

    
    ## DON'T EDIT ANYTHING BELOW THIS LINE --- I HAD TO
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[x] for x in np.squeeze(y[0])]))
    #print(' '.join([y_id_to_word[np.argmax(x)] for x in y[0]]))

final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 15)                0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 15, 128)           25600     
_________________________________________________________________
bidirectional_5 (Bidirection (None, 64)                74112     
_________________________________________________________________
repeat_vector_4 (RepeatVecto (None, 21, 64)            0         
_________________________________________________________________
bidirectional_6 (Bidirection (None, 21, 64)            49536     
_________________________________________________________________
time_distributed_10 (TimeDis (None, 21, 345)           22425     
_________________________________________________________________
activation_10 (Activation)   (None, 21, 345)           0         
Total para

## Submission
When you are ready to submit your project, do the following steps:
1. Ensure you pass all points on the [rubric](https://review.udacity.com/#!/rubrics/1004/view).
2. Submit the following in a zip file.
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.